In [ ]:
from feature_extraction import *
from visualization import *
from plotting import *
from tif_to_stack import *
from class_reordering import *
from color_masks import *
from prediction import *
import os

# Welcome to the Jupyter Notebook for the utilization of UTILE-Oxy

With this notebook, you will be guided through the code and make it possible for you to analyze your VRFB tomographies automatically.

This notebook should help you to run the automated segmentation of the cells in your images and afterward to apply the diverse functions to extract the information of interest from your data and visualize the results.

Already integrated functions are:

- Membrane separation
- Density maps for 2D bubble distribution analysis from different planes
- Individual bubble shape analysis and plot of the aspects of interest
- Membrane blocking bubbles calculation and visualization

You can provide your own data and even your own model and be able to employ those functions as well! The dataset employed for model training and the trained model are accessible at Zenodo: https://doi.org/10.5281/zenodo.11547023. 


In [ ]:
#Give a name to your project

case_name = "example1"

#First, we need to specify the folder where the tomograph slices (or tif stack) are stored

image_path= "./path/to/image.tif"

#Secondly, we need to specify where is your model stored (.hdf5)

model_path = "./path/to/model.hdf5"

#It is also required to create a folder to store the predicted masks

os.makedirs(f"./{case_name}/mask_folder", exist_ok=True)
mask_folder = f"./{case_name}/mask_folder/"

In [ ]:
#Then we call the function prediction in order to predict the classes in all images and store the masks in a separate folder

if os.path.isfile(image_path):
    save_tif_stack_as_individual_images(image_path, case_name)
    image_path = f"./{case_name}/slices"
    prediction(image_path, model_path, mask_folder)
else:
    prediction(image_path, model_path, mask_folder)

#Now we collect all the predicted images into 1 tif stack

stack_name = f"./{case_name}/{case_name}_pred.tif"   #Name of the stack
create_tiff_stack(mask_folder, stack_name)


In [ ]:
#We can directly visualize the 2D bubble densities in the three different planes

plot_densities(stack_name, case_name)

In [ ]:
# Let's give a look at that segmented volume!

volume = tifffile.imread(stack_name) #load the stack

#Select if you want to analyse just one electrode or the whole cell

left_volume, right_volume = separate_volume(volume)
volume_of_interest = right_volume # or right volume or volume for the whole cell

#Clean the desired volume

cleaned_volume = clean_volume(volume_of_interest)

#Visualize the segmentation results in 3D

visualize_volume(cleaned_volume,case_name, False) #set True if you want to record a 360° gif of your volume 


In [ ]:
#Now we can proceed with the analysis of the volume

bubble_ratio(cleaned_volume) #This function calculates the bubble voxels, background voxels, and their ratio. The results are then printed.


In [ ]:
#Individual shape analysis of the bubbles. Here a further cleaning pipeline is employed. The different bubble analyses are performed and stored in a CSV file for further plotting and visualization

filtered_volume, membrane_coords = individual_analysis(cleaned_volume, case_name)
csv_file = f"./{case_name}/output_{case_name}.csv"

#If you have already analysed your volume, you can load the filtered_volume with this function:
#filtered_volume = np.load("./path/to/your/volume.npy")

#Let's visualize the bubbles with individual colors

print(len(np.unique(filtered_volume)- 1))
visualize_labeled_volume(filtered_volume, len(np.unique(filtered_volume) - 1), case_name, False)

In [ ]:
#We can also visualize each calculated property and color-map the bubbles according to the measured value
#Uncomment the property that you want to visualize

csv_file = f"./{case_name}/output_{case_name}.csv"

visualize_property("closest_distance", filtered_volume, csv_file, case_name,side="left", capture = False) #Under side give left, right or whole depending on which part of the volume you are interested
#visualize_property("volume", filtered_volume, csv_file, case_name, side="whole", capture = False)
#visualize_property("sphericity", filtered_volume, csv_file,case_name, side="whole", capture = False)
#visualize_property("orientation", filtered_volume, csv_file,case_name, side ="right", capture = False)
#visualize_property("elongation", filtered_volume, csv_file,case_name, side ="whole", log=True,capture = False) #Log decides if you want to employ the logarithm of the number to color map the volume (recommended for features that spans over a large range of values)
#visualize_property("flatness", filtered_volume, csv_file,case_name, side ="whole", log=True, capture = False)

In [ ]:
#We can also plot the histograms and diagrams that are calculated in the csv_file

plot_properties(csv_file, case_name)

In [ ]:
#Visualization of and calculation of the membrane-blocking bubbles

blocking_voxel = membrane_block_visualization(cleaned_volume, filtered_volume, case_name,capture = False)